In [1]:
#import dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#user this code to download module
#!pip install keras-tuner

In [3]:
#load in file and create DataFrame
charity_df = pd.read_csv("charity_data_copy.csv")
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
#drop EIN and NAME columns
target = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["EIN", "NAME", "IS_SUCCESSFUL"], axis = 1)

In [5]:
#create a list of apps that are below 500 data points, which will be considered 'other'
replaced_apps = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

#created loop to replace app with the value 'other'
for app in replaced_apps:
    X['APPLICATION_TYPE'] = X['APPLICATION_TYPE'].replace(app, 'other')

#check this is done correctly 
X['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
#create a list to replace values with low amounts of data points. Anything below 300 data points is considered other.
replaced_class = ['C1700', 'C4000', 'C7000', 'C5000',
       'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230',
       'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800',
       'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210',
       'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0',
       'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600',
       'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700',
       'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732',
       'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']
for value in replaced_class:
    X['CLASSIFICATION'] = X['CLASSIFICATION'].replace(value, 'other')

#check
X['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
X = pd.get_dummies(X)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
#create train and test spilts for model
X_train, X_test, y_train, y_test = train_test_split(X, target, random_state=42)

#create scaler instance
scaler = StandardScaler()
#train scaler
scaler.fit(X_train)

#transform X datasets with scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
#Create a neural network model by assigning the number of input features and nodes for each layer using TensorFlow and Keras.
import tensorflow as tf

model = tf.keras.Sequential()

#first hidden layer
model.add(tf.keras.layers.Dense(units=100, input_dim=43, activation="relu"))

#second hidden layer
model.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

#third hidden layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#checking the structure of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4400      
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 10)                310       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs= 100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5832 - accuracy: 0.7139
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7286
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7312
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7311
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accura

In [14]:
#Evaluate the model using the test data to determine the loss and accuracy.
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Model Loss: {model_loss}, Model Accuracy: {model_accuracy}')

268/268 - 1s - loss: 0.5646 - accuracy: 0.7270 - 512ms/epoch - 2ms/step
Model Loss: 0.5645741820335388, Model Accuracy: 0.7269970774650574


In [15]:
#Save and export results to an HDF5 file
import h5py
h5py.File('AlphabetSoupCharity_Optimization.h5', 'w')

<HDF5 file "AlphabetSoupCharity_Optimization.h5" (mode r+)>